<a href="https://colab.research.google.com/github/ced-sys/.py/blob/main/CUDA_Fluid_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Oct 25 06:48:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   38C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install pycuda pillow numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 10.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=8eae3274f9f55974a1bd2cb130836ffefa117c33768840228f8e2504cbf4a075
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda


In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import time

In [4]:
WIDTH=256
HEIGHT=256
DT=0.1
DIFFUSION=0.0001
VISCOSITY=0.0001

print(f"Creating a {WIDTH}x{HEIGHT} smoke simulation...")
print(f"Total pixels to simulate: {WIDTH*HEIGHT}")

Creating a 256x256 smoke simulation...
Total pixels to simulate: 65536


In [6]:
cuda_code="""
//Add smoke density at a location
__global__ void add_source(float *field, float*source, int size, float dt){
  int idx=blockIdx.x * blockDim.x+threadIdx.x;
  if (idx < size){
    field[idx]+=dt*source[idx];
  }
}

//Advection: Move smoke based on velocity
__global__ void advect(float *d, float *d0, float *u, float *v, int width, int height, float dt){
  int x=blockIdx.x * blockDim.x+threadIdx.x;
  int y=blockIdx.y* blockDim.y+threadIdx.y;

  if (x>=width || y>= height) return;

  int idx=y*width+x;

  //Trace back in time to find where this smoke came from
  float x_prev=x-dt*u[idx];
  float y_prev=y-dt*v[idx];

  //Clamp to grid boundaries
  if (x_prev <0.5f) x_prev=0.5f;
  if (x_prev > width-1.5f) x_prev=width-1.5f;
  if (y_prev<0.5f) y_prev=0.5f;
  if (y_prev>height-1.5f) y_prev=height-1.5f;

  //Bilinear interpolation (smooth sampling between grid points)
  int i0=(int)x_prev;
  int i1=i0+1;
  int j0=(int)y_prev;
  int j1=j0+1;

  float s1=x_prev-i0;
  float s0=1.0f-s1;
  float t1=y_prev-j0;
  float t0=1.0f-t1;

  //Weighted average of 4 nearby points
  d[idx]=s0*(t0*d0[j0*width+i0]+t1*d0[j1*width+i0])+s1*(t0*d0[j0*width+i1]+t1*d0[j1*width+i1]);
}

//Diffusion: Make smoke spread out smoothly (Jacobi iteration)
__global__ void diffuse(float *x, float *x0, float diff, float dt, int width, int height){
  int i=blockIdx.x*blockDim.x+threadIdx.x;
  int j=blockIdx.y*blockDim.y+threadIdx.y;

  if (i>=width|| j>=height) return;
  if(i==0 || i==width-1 || j==0 || j==height-1) return;

  int idx=j*width+i;
  float a=dt*diff*width*height;

  //Average with neighbours, weighted by diffusion
  x[idx]=(x0[idx]+a*(x[idx-1]+x[idx+1]+ x[idx-width]+x[idx+width]))/(1+4*a);
}

//Fade out smoke over time
__global__ void fade(float *field, int size, float fade_rate){
  int idx=blockIdx.x*blockDim.x+threadIdx.x;
  if (idx<size){
    field[idx]*= fade_rate;
  }
}
"""

mod=SourceModule(cuda_code)

#Get references to our kernels
add_source_kernel=mod.get_function("add_source")
advect_kernel=mod.get_function("advect")
diffuse_kernel=mod.get_function("diffuse")
fade_kernel=mod.get_function("fade")

print('Kernels compiled successfully!')

Kernels compiled successfully!
